# 등급예측 일반 데이터 분석

### 데이터 읽어오기

In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline
#from sklearn.metrics import classification_report,confusion_matrix
from google.colab import files #데이터 불러오기
myfile=files.upload()


Saving 농가전체(등급).csv to 농가전체(등급).csv


In [2]:
dataset=pd.read_csv('농가전체(등급).csv')

dataset.loc[dataset['등급']=='등외', '등급'] = 3
dataset.loc[dataset['등급']=='1+', '등급'] = 0

dataset['등급'] = pd.to_numeric(dataset['등급'])
dataset.dtypes

농장명       object
농장주       object
작업장명      object
판정일        int64
도체번호       int64
판정방법      object
도체형태      object
성         object
도체중        int64
등지방두께      int64
이력번호     float64
등급         int64
dtype: object

In [3]:
from sklearn.preprocessing import LabelEncoder

encoding_list=['농장명','농장주','작업장명','판정방법','도체형태','성']
for i in encoding_list:
    encoder = LabelEncoder()
    encoder.fit(dataset[i])
    labels = encoder.transform(dataset[i])
    dataset[i]=labels
    print('인코딩 클래스:',encoder.classes_)

인코딩 클래스: ['덕치' '두원/미곡' '멧골' '산전' '유진' '재승' '진해' '큰부자']
인코딩 클래스: ['권순태' '김인재' '김재관' '박명석' '박종배' '엄익붕' '이종구' '최윤호']
인코딩 클래스: ['(농주)경기엘피씨' '(유)금돈' '(유)참푸른글로벌' '(주)광축' '(주)영남엘피씨' '(주)우경축산' '(주)중앙축산'
 '(주)친환경명품축산' '(주)팜스코' '(주)팜스토리 한냉' '강원엘피씨' '김해축공' '논산계룡축협' '농협김제목우촌'
 '농협나주' '농협음성' '농협포크빌' '대산푸드' '대성실업' '대원축산(주)' '동아식품' '맥우' '박달재LPC' '백제나루'
 '부경축공' '사조산업' '사조산업충주공장' '삼성식품(주)' '삼호축산' '서라벌' '신흥산업' '알파미트엘피씨' '오성식품'
 '우진산업' '장원식품' '제일리버스' '중앙산업' '철원한양' '축림' '평창기업' '포천농축산' '함평축산물종합처리장'
 '화정식품']
인코딩 클래스: ['온']
인코딩 클래스: ['박피' '탕박']
인코딩 클래스: ['거세' '수' '암']


In [4]:
dataset=dataset.sort_values(by='등급')
dataset=dataset.reset_index()
del dataset['index']

dataset.dtypes

농장명        int64
농장주        int64
작업장명       int64
판정일        int64
도체번호       int64
판정방법       int64
도체형태       int64
성          int64
도체중        int64
등지방두께      int64
이력번호     float64
등급         int64
dtype: object

In [5]:
data=dataset.sort_values(by='등급')

#y값의 비율 맞춰서 추출
len0=len(data[data['등급']==0])
len1=len(data[data['등급']==1])
len2=len(data[data['등급']==2])
len3=len(data[data['등급']==3])

import random
random.seed(2021)
ts_idx=random.sample(range(0,len0-1),int(len0*0.3))
ts_idx.extend(random.sample(range(len0,len0+len1-1),int(len1*0.3)))
ts_idx.extend(random.sample(range(len0+len1,len0+len1+len2-1),int(len2*0.3)))
ts_idx.extend(random.sample(range(len0+len1+len2,len0+len1+len2+len3-1),int(len3*0.3)))
ts_idx.sort()

tr_idx=list(range(0,len(data)-1))
for i in range(0,len(data)-1):
    if i in ts_idx:
        tr_idx.remove(i)

X_train = data.loc[tr_idx,['농장명','농장주','작업장명','판정일','도체번호','판정방법','도체형태','성','도체중','등지방두께','이력번호']].values
X_test = data.loc[ts_idx,['농장명','농장주','작업장명','판정일','도체번호','판정방법','도체형태','성','도체중','등지방두께','이력번호']].values
y_train = data.loc[tr_idx,['등급']].values
y_test = data.loc[ts_idx,['등급']].values

print("train set 개수 = ", len(y_train))
print("test set 개수 = ", len(y_test))

train set 개수 =  97433
test set 개수 =  41756


### Logistic Regression

1. sklearn의 LogisticRegression 모델

In [ ]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression(solver='liblinear',multi_class='ovr')

logmodel.fit(X_train,y_train)

from sklearn.model_selection import StratifiedKFold, cross_val_score
skf=StratifiedKFold(n_splits=3,shuffle=True, random_state=0)
score=cross_val_score(logmodel,X_train,y_train,cv=skf)
print("forest StratifiedKFold 정확도: ", score.mean())


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

forest StratifiedKFold 정확도:  0.34087013657857873


In [ ]:
#predicting the values
y_pred = logmodel.predict(X_test)

#confusion matrix
from sklearn.metrics import confusion_matrix
cm_Log = confusion_matrix(y_test, y_pred)
print(cm_Log)

#Accuracy and report of the classifier
from sklearn.metrics import accuracy_score
pred_acc=accuracy_score(y_pred, y_test)
print("예측정확도: ", pred_acc)

#report
#report_Log = classification_report(y_test, y_pred)
#print(report_Log)

[[    0 13034     0     0]
 [    0 14233     0     0]
 [    0 12842     0     0]
 [    0  1647     0     0]]
예측정확도:  0.34086119360091965


2. statsmodels.api의 Logit 모델
> 에러 어떻게 해결하는거지....?

In [6]:
import statsmodels.api as sm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [28]:
tr_data=data.loc[tr_idx,:]
ts_data=data.loc[ts_idx,:]

In [24]:
train_cols=data.columns[:11]
train_cols

Index(['농장명', '농장주', '작업장명', '판정일', '도체번호', '판정방법', '도체형태', '성', '도체중',
       '등지방두께', '이력번호'],
      dtype='object')

In [25]:
logtit_model=sm.Logit(tr_data['등급'],tr_data[train_cols])


ValueError: ignored

In [29]:
logtit_model=sm.Logit(y_train,X_train)

ValueError: ignored

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()

dtree.fit(X_train,y_train)

from sklearn.model_selection import StratifiedKFold
skf=StratifiedKFold(n_splits=3,shuffle=True, random_state=0)
score=cross_val_score(dtree,X_train,y_train,cv=skf)
print("forest StratifiedKFold 정확도: ", score.mean())


forest StratifiedKFold 정확도:  0.9138792877653055


In [ ]:
#예측
result = dtree.predict(X_test)

#예측정확도
from sklearn.metrics import accuracy_score
pred_acc=accuracy_score(result, y_test)
print("예측정확도: ", pred_acc)
    

예측정확도:  0.9196762141967622


### RamdomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=200)

rfc.fit(X_train, y_train)

from sklearn.model_selection import StratifiedKFold, cross_val_score
skf=StratifiedKFold(n_splits=3,shuffle=True, random_state=0)
score=cross_val_score(rfc,X_train,y_train,cv=skf)
print("forest StratifiedKFold 정확도: ", score.mean())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Plea

forest StratifiedKFold 정확도:  0.9448030706939988


In [ ]:
#predicting the values
y_pred = rfc.predict(X_test)

#confusion matrix
from sklearn.metrics import confusion_matrix
cm_RF = confusion_matrix(y_test, y_pred)
print(cm_RF)

#예측정확도
from sklearn.metrics import accuracy_score
pred_acc=accuracy_score(y_pred, y_test)
print("예측정확도: ", pred_acc)

#Report
#report_RF = classification_report(y_test, y_pred)
#print(report_RF)

[[12805   228     1     0]
 [ 1308 12857    68     0]
 [   23   546 12271     2]
 [    1     1    15  1630]]
예측정확도:  0.9474806015901907


In [ ]:
1878/(1878+55)

0.9715468184169684

In [ ]:
2061/(2061+211)

0.9071302816901409

In [ ]:
2021/(2021+87)

0.9587286527514232

In [ ]:
241/(241+15)

0.94140625

In [ ]:
y_testt=[]
for i in range(0,len(y_test)):
  y_testt.extend(y_test[i])

y_testt

import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/')
%pwd
pd.concat([pd.DataFrame(y_testt),pd.DataFrame(y_pred)],axis=1).to_excel("sample.xlsx")

Mounted at /content/drive


### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=34, metric='manhattan',weights='distance')

knn.fit(X_train,y_train)

from sklearn.model_selection import StratifiedKFold
skf=StratifiedKFold(n_splits=10,shuffle=True, random_state=0)
score=cross_val_score(knn,X_train,y_train,cv=skf)
print("forest StratifiedKFold 정확도: ", score.mean())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. P

forest StratifiedKFold 정확도:  0.5365430985186282


In [ ]:
#predicting the values
y_pred = knn.predict(X_test)

#confusion matrix
cm_KNN = confusion_matrix(y_test, y_pred)
print(cm_KNN)

#Accuracy
from sklearn.metrics import accuracy_score
pred_acc=accuracy_score(y_pred, y_test)
print("예측정확도: ", pred_acc)

#Report
#report_KNN = classification_report(y_test, y_pred)
#print(report_KNN)

[[8680 3723  629    2]
 [5617 6854 1759    3]
 [2746 4104 5983    9]
 [ 104  113  136 1294]]
예측정확도:  0.5462927483475428


### Naive Bayes Classifier

1. 원래 코드

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()

classifier.fit(X_train,y_train)

from sklearn.model_selection import StratifiedKFold, cross_val_score
skf=StratifiedKFold(n_splits=3,shuffle=True, random_state=0)
score=cross_val_score(classifier,X_train,y_train,cv=skf)
print("forest StratifiedKFold 정확도: ", score.mean())

#predicting the values
y_pred = classifier.predict(X_test)

#confusion matrix
from sklearn.metrics import confusion_matrix
cm_NB = confusion_matrix(y_test, y_pred)
print(cm_NB)

#Accuracy
from sklearn.metrics import accuracy_score
pred_acc=accuracy_score(y_pred, y_test)
print("예측정확도: ", pred_acc)
#0.79

#Report
#report_NB = classification_report(y_test, y_pred)
#print(report_NB)


/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = col

forest StratifiedKFold 정확도:  0.34087013657857873
[[    0 13034     0     0]
 [    0 14233     0     0]
 [    0 12842     0     0]
 [    0  1647     0     0]]
예측정확도:  0.34086119360091965


2. 새로 찾은 코드 (0115)
> 정확도가 더 떨어짐.

In [39]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

classifier = MultinomialNB()
classifier.fit(X_train, y_train)
pred=classifier.predict(X_train)
print("훈련 정확도: ", accuracy_score(y_train,pred))

y_pred=classifier.predict(X_test)
print("테스트 정확도: ", accuracy_score(y_test,y_pred))

#confusion matrix
from sklearn.metrics import confusion_matrix
cm_NB = confusion_matrix(y_test, y_pred)
print(cm_NB)

훈련 정확도:  0.22018207383535351
테스트 정확도:  0.22054315547466233
[[4173   62 3760 5039]
 [4259   80 4236 5658]
 [3402   56 4155 5229]
 [ 436   15  395  801]]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
classifier = GradientBoostingClassifier()

classifier.fit(X_train,y_train)

from sklearn.model_selection import StratifiedKFold
skf=StratifiedKFold(n_splits=3,shuffle=True, random_state=0)
score=cross_val_score(classifier,X_train,y_train,cv=skf)
print("forest StratifiedKFold 정확도: ", score.mean())

#predicting the values
y_pred = classifier.predict(X_test)

#confusion matrix
cm_GBC = confusion_matrix(y_test, y_pred)
print(cm_GBC)

#Accuracy
from sklearn.metrics import accuracy_score
pred_acc=accuracy_score(y_pred, y_test)
print("예측정확도: ", pred_acc)
#0.8816666666666667

#Report
#report_GBC = classification_report(y_test, y_pred)
#print(report_GBC)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 

forest StratifiedKFold 정확도:  0.9476255218877286
[[13028     6     0     0]
 [ 1436 12796     1     0]
 [   26   591 12223     2]
 [    1     1    16  1629]]
예측정확도:  0.950186799501868


### SVM

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0, C = 1)

classifier.fit(X_train,y_train)

#print(classifier.support_vectors_)
#결정경계를 정의하는 서포트 벡터 확인

from sklearn.model_selection import StratifiedKFold
skf=StratifiedKFold(n_splits=10,shuffle=True, random_state=0)
score=cross_val_score(classifier,X_train,y_train,cv=skf)
print("forest StratifiedKFold 정확도: ", score.mean())



/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#predicting the values
y_pred = classifier.predict(X_test)

#confusion matrix
cm_SVC = confusion_matrix(y_test, y_pred)
print(cm_SVC)

#Accuracy
from sklearn.metrics import accuracy_score
pred_acc=accuracy_score(y_pred, y_test)
print("예측정확도: ", pred_acc)


#Report
#report_SVC = classification_report(y_test, y_pred)
#print(report_SVC)